# **Import packages**

In [ ]:
import torch
import json
import random
from math import *
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
import matplotlib.pyplot as plt
import os

 **Import data**

In [ ]:
TRANSACTIONS_PATH = 'TRANSACTIONS_PATH'
TAXONOMY_PATH = 'TAXONOMY_PATH'
PRODUCTS_PATH = 'PRODUCTS_PATH'

PRETRAIN_PATH = 'PRETRAIN_PATH'
TRAIN_PATH = 'TRAIN_PATH'
TEST_PATH = 'TEST_PATH'

PRETRAIN_CLN_PATH = 'PRETRAIN_CLN_PATH'
TRAIN_CLN_PATH = 'TRAIN_CLN_PATH'

In [ ]:
df_org = pd.read_csv(TRANSACTIONS_PATH)
df_taxonomy = pd.read_csv(TAXONOMY_PATH, delimiter =";")

In [ ]:
f = open(PRODUCTS_PATH)
product_data = json.load(f)

In [ ]:
df_org['prod_series_cln'] = df_org.prod_series.str.replace(';',' ')
product_IDS = set([item for sublist in (df_org['prod_series_cln'].str.split(" ").tolist()) for item in sublist])
products = len(product_IDS)

**Create Product ID to name dictionary**

In [ ]:
#create Json file with product ID: product name

ids_to_names = dict()
for product_ID in product_IDS:
  product_name = df_taxonomy[df_taxonomy['OnlineRetailItemNbr']==int(product_ID)]['lowest_level'].unique()
  s = product_name.tolist()
  if (type(s[0]) == float):
    print(s)
    continue
  if len(s) > 1:
    if (type(s[1]) == float):
      print(s)
      continue
  str1 = ''
  A = str1.join(s)
  ids_to_names[product_ID] = A

import json
with open('productID_tonames.json', 'w') as fp:
    json.dump(ids_to_names, fp)

## **Clean Less than 10 Purchases**

In [ ]:
#check how many products are truncated
basket_size = []
for basket in df_org['prod_series_cln']:
  bas_size = len(basket.split())
  basket_size.append(bas_size)

basket_test = np.array(basket_size)
print(np.count_nonzero(np.where(basket_test>=60))/len(basket_test))

In [ ]:
# create product count dictionary
baskets = list(df_org['prod_series_cln'])
product_cnt = {}

for basket in tqdm(baskets):
  items = basket.split()
  for item in items:
    if item not in product_cnt:
      product_cnt[item] = 1
    else:
      product_cnt[item] += 1

100%|██████████| 4310165/4310165 [00:45<00:00, 93842.44it/s] 


In [ ]:
# create dataset with items sold less than 10 times removed
freq_cnt = 0
del_items = []

for k,v in product_cnt.items():
  if v>10:
    freq_cnt+=1
  else:
    del_items.append(k)

baskets_cln = []

for basket in tqdm(baskets):
  items = basket.split()
  #print(items)
  if items != None:
    for delete in del_items:
      if delete in items:
        items.remove(str(delete))
        baskets_cln.append(' '.join(items))

26623

In [ ]:
df_validIDs = pd.DataFrame( df_taxonomy_paths.iloc[:,0].unique(), columns = ['product_ID'])
set_validIDs = set(df_validIDs['product_ID'].astype(str).tolist())
df_org['prod_series_cln'] = df_org.prod_series.str.replace(';',' ')
df_org['basket_length'] = df_org.prod_series_cln.apply(lambda x: x.count(" ") + 1)

def return_last_item(x):
    list_basket = str(x).split(' ')
    last_item = list_basket[-1]
    return last_item

df_org['last product'] = df_org['prod_series_cln'].apply(return_last_item)

In [ ]:
#histogram of product IDs in basket

plt.hist(df_org['basket_length'], bins=100, range = (1,100))
plt.axvline(df_org['basket_length'].mean(), color='black', linestyle='dashed', linewidth=0.5, label="Mean=17")
plt.xlabel("Number of items in basket")
plt.ylabel("Frequency")
plt.legend(loc='upper right')
plt.show()
plt.savefig('basketsize.jpg')

Write sample files for pre-training, training and test

In [ ]:
#sample sizes
num_pretrain = 3750000
num_train    = 100000
num_test     =  25000

#randomise order of data
random.seed(10)
s_rand = df_org['prod_series_cln'].iloc[np.random.permutation(len(df_org))]

In [ ]:
#write text files
txt_file = open(PRETRAIN_PATH, "w")
for element in s_rand[:num_pretrain]:
    txt_file.write(element + "\n")
txt_file.close()

txt_file = open(TRAIN_PATH, "w")
for element in s_rand[num_pretrain+1:num_pretrain+num_train]:
    txt_file.write(element + "\n")
txt_file.close()

txt_file = open(TEST_, "w")
for element in s_rand[num_pretrain+num_train+1:num_pretrain+num_train+num_test]:
    txt_file.write(element + "\n")
txt_file.close()